In [1]:
import plotly.express as px


In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
from collections import OrderedDict
import argparse
import re
from scipy.stats import gmean

pattern_map = {
    'runtime': 'simTicks\s+\d+',
    'network_lat': 'system.ruby.network.average_packet_network_latency\s+\d+',
    'inst': 'simInsts\s+\d+',
    'd_rd_access': 'system.switch_cpus\d+.mmu.dtb.rdAccesses\s+\d+',
    'i_rd_access': 'system.switch_cpus\d+.mmu.itb.rdAccesses\s+\d+',
    'd_wr_access': 'system.switch_cpus\d+.mmu.dtb.wrAccesses\s+\d+',
    'i_wr_access': 'system.switch_cpus\d+.mmu.itb.wrAccesses\s+\d+',
    'd_rd_miss': 'system.switch_cpus\d+.mmu.dtb.rdMisses\s+\d+',
    'i_rd_miss': 'system.switch_cpus\d+.mmu.itb.rdMisses\s+\d+',
    'd_wr_miss': 'system.switch_cpus\d+.mmu.dtb.wrMisses\s+\d+',
    'i_wr_miss': 'system.switch_cpus\d+.mmu.itb.wrMisses\s+\d+',
    
}

prune_map = {
    'runtime': lambda line: float(line.replace(' ','').split('simTicks')[-1].split('#')[0]),
    'network_lat': lambda line: float(line.replace(' ','').split('system.ruby.network.average_packet_network_latency')[-1].split('(')[0]),
    'inst': lambda line: float(line.replace(' ','').split('simInsts')[-1].split('#')[0]),
    'd_rd_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'i_rd_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'd_wr_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'i_wr_access': lambda line: float(line.replace(' ','').split('Accesses')[-1].split('#')[0]),
    'd_rd_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'i_rd_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'd_wr_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
    'i_wr_miss': lambda line: float(line.replace(' ','').split('Misses')[-1].split('#')[0]),
}


In [4]:
outdir = [\
    '20220608140651/gem5/my_STATS/base/blackscholes_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/blackscholes_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/blackscholes_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/bodytrack_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/bodytrack_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/bodytrack_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/canneal_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/canneal_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/canneal_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/dedup_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/dedup_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/dedup_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/ferret_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/ferret_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/ferret_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/fluidanimate_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/fluidanimate_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/fluidanimate_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/freqmine_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/freqmine_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/freqmine_simsmall_64_2/stats.txt', \
#     '20220608140651/gem5/my_STATS/base/streamcluster_simsmall_64_0/stats.txt', \
#     '20220608140651/gem5/my_STATS/base/streamcluster_simsmall_64_1/stats.txt', \
#     '20220608140651/gem5/my_STATS/base/streamcluster_simsmall_64_2/stats.txt', \
    #'20220608140651/gem5/my_STATS/base/swaptions_simsmall_64_0/stats.txt', \
    #'20220608140651/gem5/my_STATS/base/swaptions_simsmall_64_1/stats.txt', \
    #'20220608140651/gem5/my_STATS/base/swaptions_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/raytrace_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/raytrace_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/raytrace_simsmall_64_2/stats.txt', \
    '20220608140651/gem5/my_STATS/base/vips_simsmall_64_0/stats.txt', \
    '20220608140651/gem5/my_STATS/base/vips_simsmall_64_1/stats.txt', \
    '20220608140651/gem5/my_STATS/base/vips_simsmall_64_2/stats.txt', \


    ]


In [5]:
def get_list(file, metric):
    pattern = pattern_map[metric]

    list_ = []
    with open(file, 'r') as searchfile:
        for line in searchfile:
            if re.match(pattern, line):
                prune_func = prune_map[metric]
                res = prune_func(line)
                list_.append(res)
    # print(list_)
    return list_


In [6]:
o_arr = []
for o in outdir:
    name=o.split('/')[4].split('_')[0]
    if name not in o_arr: o_arr.append(name)
o_arr

['blackscholes',
 'bodytrack',
 'canneal',
 'dedup',
 'ferret',
 'fluidanimate',
 'freqmine',
 'raytrace',
 'vips']

In [7]:
metric = 'runtime'
rt_list_base = [get_list(o, metric)[0] for o in outdir if '_1' in o]
print(f'rt base ={rt_list_base}')
rt_list_quater = [get_list(o, metric)[0] for o in outdir if '_0' in o]
print(f'rt quater={rt_list_quater}')
rt_list_red = [get_list(o, metric)[0] for o in outdir if '_2' in o]
print(f'rt red={rt_list_red}\n')

rt_norm_quater = [q / b for b,q in zip(rt_list_base, rt_list_quater)]
rt_norm_mean_quater = gmean(rt_norm_quater)
print(f'rt (norm) quater ={rt_norm_quater}')

rt_norm_red = [q / b for b,q in zip(rt_list_base, rt_list_red)]
rt_norm_mean_red = gmean(rt_norm_red)
print(f'rt (norm) red ={rt_norm_red}')

rt base =[17407341001.0, 211459828001.0, 78264582001.0, 465658400001.0, 775687704001.0, 155091224001.0, 1178362106501.0, 152131580501.0, 514178573001.0]
rt quater=[17476704001.0, 211362545001.0, 82473845001.0, 482080214001.0, 783924837501.0, 151957263001.0, 1219779333001.0, 153007647001.0, 514996066501.0]
rt red=[17309436001.0, 209926699501.0, 81980078501.0, 478151071001.0, 789865232501.0, 156419568501.0, 1214155376001.0, 152054995501.0, 512766740501.0]

rt (norm) quater =[1.003984698179694, 0.9995399457148922, 1.0537824759601504, 1.0352657956990892, 1.0106191363579864, 0.979792789565064, 1.0351481316918645, 1.00575861038921, 1.0015899019191499]
rt (norm) red =[0.9943756487567874, 0.9927497883901014, 1.047473536624172, 1.0268279730376886, 1.0182773665572784, 1.0085649236993026, 1.030375441727572, 0.9994965871007992, 0.9972541981052228]


In [8]:
import plotly.graph_objects as go

o_arr.append('geomean')
rt_norm_quater.append(rt_norm_mean_quater)
rt_norm_red.append(rt_norm_mean_red)

fig = go.Figure(data=[
    go.Bar(name='quater', x=o_arr, y=rt_norm_quater),
    go.Bar(name='fastAccess', x=o_arr, y=rt_norm_red)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()


In [9]:
#//////// rd
metric = 'd_rd_access'
d_rd_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd rd access base ={d_rd_access_base}')
d_rd_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd rd access quater={d_rd_access_quater}')
d_rd_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd rd access red={d_rd_access_red}\n')

metric = 'd_rd_miss'
d_rd_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd rd miss base ={d_rd_miss_base}')
# d_rd_mr_base = [float(m/a)*100 for m,a in zip(d_rd_miss_base, d_rd_access_base)]
# print(f'r rd mr base={d_rd_mr_base}')

d_rd_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd rd miss quater={d_rd_miss_quater}')
# d_rd_mr_quater = [float(m/a)*100 for m,a in zip(d_rd_miss_quater, d_rd_access_quater)]
# print(f'r rd mr quater={d_rd_mr_quater}')

d_rd_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd rd miss red={d_rd_miss_red}\n')
# d_rd_mr_red = [float(m/a)*100 for m,a in zip(d_rd_miss_red, d_rd_access_red)]
# print(f'r rd mr red={d_rd_mr_red}')

#//////// wr
metric = 'd_wr_access'
d_wr_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd wr access base ={d_wr_access_base}')
d_wr_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd wr access quater={d_wr_access_quater}')
d_wr_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd wr access red={d_wr_access_red}\n')

metric = 'd_wr_miss'
d_wr_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd wr miss base ={d_wr_miss_base}')
# d_wr_mr_base = [float(m/a)*100 for m,a in zip(d_wr_miss_base, d_wr_access_base)]
# print(f'r wr mr base={d_wr_mr_base}')

d_wr_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd wr miss quater={d_wr_miss_quater}')
# d_wr_mr_quater = [float(m/a)*100 for m,a in zip(d_wr_miss_quater, d_wr_access_quater)]
# print(f'r wr mr quater={d_rd_wr_quater}')

d_wr_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd wr miss red={d_wr_miss_red}\n')
# d_wr_mr_red = [float(m/a)*100 for m,a in zip(d_wr_miss_red, d_wr_access_red)]
# print(f'r wr mr red={d_wr_mr_red}')


In [10]:
#//////// rd
metric = 'i_rd_access'
i_rd_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'i rd access base ={i_rd_access_base}')
i_rd_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'i rd access quater={i_rd_access_quater}')
i_rd_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'i rd access red={i_rd_access_red}\n')

metric = 'i_rd_miss'
i_rd_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'i rd miss base ={i_rd_miss_base}')
# i_rd_mr_base = [float(m/a)*100 for m,a in zip(i_rd_miss_base, i_rd_access_base)]
# print(f'i rd mr base={i_rd_mr_base}')

i_rd_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'i rd miss quater={i_rd_miss_quater}')
# i_rd_mr_quater = [float(m/a)*100 for m,a in zip(i_rd_miss_quater, i_rd_access_quater)]
# print(f'i rd mr quater={i_rd_mr_quater}')

i_rd_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd rd miss red={i_rd_miss_red}\n')
# i_rd_mr_red = [float(m/a)*100 for m,a in zip(i_rd_miss_red, i_rd_access_red)]
# print(f'i rd mr red={i_rd_mr_red}')

#//////// wr
metric = 'i_wr_access'
i_wr_access_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'd wr access base ={i_wr_access_base}')
i_wr_access_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'd wr access quater={i_wr_access_quater}')
i_wr_access_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'd wr access red={i_wr_access_red}\n')

metric = 'i_wr_miss'
i_wr_miss_base = [get_list(o, metric) for o in outdir if '_1' in o]
# print(f'i wr miss base ={i_wr_miss_base}')
# i_wr_mr_base = [float(m/a)*100 for m,a in zip(i_wr_miss_base, i_wr_access_base)]
# print(f'i wr mr base={i_wr_mr_base}')

i_wr_miss_quater = [get_list(o, metric) for o in outdir if '_0' in o]
# print(f'i wr miss quater={i_wr_miss_quater}')
# i_wr_mr_quater = [float(m/a)*100 for m,a in zip(i_wr_miss_quater, i_wr_access_quater)]
# print(f'i wr mr quater={i_rd_wr_quater}')

i_wr_miss_red = [get_list(o, metric) for o in outdir if '_2' in o]
# print(f'i wr miss red={i_wr_miss_red}\n')
# i_wr_mr_red = [float(m/a)*100 for m,a in zip(i_wr_miss_red, i_wr_access_red)]
# print(f'i wr mr red={i_wr_mr_red}')


In [11]:
# calculate hit
d_rd_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_rd_access_base, d_rd_miss_base)]
d_rd_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_rd_access_quater, d_rd_miss_quater)]
d_rd_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_rd_access_red, d_rd_miss_red)]

d_wr_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_wr_access_base, d_wr_miss_base)]
d_wr_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_wr_access_quater, d_wr_miss_quater)]
d_wr_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(d_wr_access_red, d_wr_miss_red)]

i_rd_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_rd_access_base, i_rd_miss_base)]
i_rd_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_rd_access_quater, i_rd_miss_quater)]
i_rd_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_rd_access_red, i_rd_miss_red)]

i_wr_hit_base = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_wr_access_base, i_wr_miss_base)]
i_wr_hit_quater = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_wr_access_quater, i_wr_miss_quater)]
i_wr_hit_red = [[a-m for a, m in zip(a_l, m_l)] for a_l, m_l in zip(i_wr_access_red, i_wr_miss_red)]



In [12]:
d_rd_miss_base_sum = [sum(l) for l in d_rd_miss_base]
d_rd_miss_quater_sum = [sum(l) for l in d_rd_miss_quater]
d_rd_miss_red_sum = [sum(l) for l in d_rd_miss_red]

d_rd_hit_base_sum = [sum(l) for l in d_rd_hit_base]
d_rd_hit_quater_sum = [sum(l) for l in d_rd_hit_quater]
d_rd_hit_red_sum = [sum(l) for l in d_rd_hit_red]

i_rd_miss_base_sum = [sum(l) for l in i_rd_miss_base]
i_rd_miss_quater_sum = [sum(l) for l in i_rd_miss_quater]
i_rd_miss_red_sum = [sum(l) for l in i_rd_miss_red]

i_rd_hit_base_sum = [sum(l) for l in i_rd_hit_base]
i_rd_hit_quater_sum = [sum(l) for l in i_rd_hit_quater]
i_rd_hit_red_sum = [sum(l) for l in i_rd_hit_red]


d_wr_miss_base_sum = [sum(l) for l in d_wr_miss_base]
d_wr_miss_quater_sum = [sum(l) for l in d_wr_miss_quater]
d_wr_miss_red_sum = [sum(l) for l in d_wr_miss_red]

d_wr_hit_base_sum = [sum(l) for l in d_wr_hit_base]
d_wr_hit_quater_sum = [sum(l) for l in d_wr_hit_quater]
d_wr_hit_red_sum = [sum(l) for l in d_wr_hit_red]

i_wr_miss_base_sum = [sum(l) for l in i_wr_miss_base]
i_wr_miss_quater_sum = [sum(l) for l in i_wr_miss_quater]
i_wr_miss_red_sum = [sum(l) for l in i_wr_miss_red]

i_wr_hit_base_sum = [sum(l) for l in i_wr_hit_base]
i_wr_hit_quater_sum = [sum(l) for l in i_wr_hit_quater]
i_wr_hit_red_sum = [sum(l) for l in i_wr_hit_red]


In [13]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2,
    subplot_titles=("data tlb read", "data tlb write", "inst tlb read", "inst tlb write"))

In [14]:
# fig1
# group1: base
fig.add_trace(
    go.Bar(
        name='hit',
        x=o_arr,
        y=d_rd_hit_base_sum, 
        offsetgroup=0,
        legendgroup="base",  # this can be any string, not just "group"
        legendgrouptitle_text="base",
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=1
)
fig.add_trace(
    go.Bar(
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_rd_miss_base_sum, 
        offsetgroup=0, base=d_rd_hit_base_sum,
        legendgroup="base",
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=1
)

# group2: quater
fig.add_trace(
    go.Bar(
        name='hit',
        x=o_arr,
        y=d_rd_hit_quater_sum, 
        offsetgroup=1,
        legendgroup="quater",
        legendgrouptitle_text="quater",
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=1
)
fig.add_trace(
    go.Bar(
        name='miss',
        marker_pattern_shape="x",
        x=o_arr,
        y=d_rd_miss_quater_sum, 
        offsetgroup=1, base=d_rd_hit_quater_sum,
        legendgroup="quater",
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=1
)

# group3: red
fig.add_trace(
    go.Bar(
        name='hit',
        x=o_arr,
        y=d_rd_hit_red_sum, 
        offsetgroup=2,
        legendgroup="fastAccess",
        legendgrouptitle_text="fastAccess",
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=1
)
fig.add_trace(
    go.Bar(
        name='miss',
        marker_pattern_shape="+",
        x=o_arr,
        y=d_rd_miss_red_sum, 
        offsetgroup=2, base=d_rd_hit_red_sum,
        legendgroup="fastAccess",
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=1
)

In [15]:
# fig2
# group1: base
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=d_wr_hit_base_sum, 
        offsetgroup=3,
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_wr_miss_base_sum, 
        offsetgroup=3, base=d_wr_hit_base_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=2
)

# group2: quater
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=d_wr_hit_quater_sum, 
        offsetgroup=4,
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_wr_miss_quater_sum, 
        offsetgroup=4, base=d_wr_hit_quater_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=2
)

# group3: red
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=d_wr_hit_red_sum, 
        offsetgroup=5,
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=1, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=d_wr_miss_red_sum, 
        offsetgroup=5, base=d_wr_hit_red_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=1, 
    col=2
)

In [16]:
# fig3
# group1: base
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_rd_hit_base_sum, 
        offsetgroup=6,
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=1
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=i_rd_miss_base_sum, 
        offsetgroup=6, base=i_rd_hit_base_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=1
)

# group2: quater
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_rd_hit_quater_sum, 
        offsetgroup=7,
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=1
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="x",
        x=o_arr,
        y=i_rd_miss_quater_sum, 
        offsetgroup=7, base=i_rd_hit_quater_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=1
)

# group3: red
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_rd_hit_red_sum, 
        offsetgroup=8,
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=1
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="+",
        x=o_arr,
        y=i_rd_miss_red_sum, 
        offsetgroup=8, base=i_rd_hit_red_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=1
)

In [17]:
# fig4
# group1: base
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_wr_hit_base_sum, 
        offsetgroup=6,
        marker_pattern_shape="",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="",
        x=o_arr,
        y=i_wr_miss_base_sum, 
        offsetgroup=6, base=i_wr_hit_base_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=2
)

# group2: quater
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_wr_hit_quater_sum, 
        offsetgroup=7,
        marker_pattern_shape="x",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="x",
        x=o_arr,
        y=i_wr_miss_quater_sum, 
        offsetgroup=7, base=i_wr_hit_quater_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=2
)

# group3: red
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='hit',
        x=o_arr,
        y=i_wr_hit_red_sum, 
        offsetgroup=8,
        marker_pattern_shape="+",
        marker=dict(color="green", line_color="green", pattern_fillmode="overlay")
    ), 
    row=2, 
    col=2
)
fig.add_trace(
    go.Bar(
        showlegend=False,
        name='miss',
        marker_pattern_shape="+",
        x=o_arr,
        y=i_wr_miss_red_sum, 
        offsetgroup=8, base=i_wr_hit_red_sum,
        marker=dict(color="red", line_color="red", pattern_fillmode="overlay")
        
    ), 
    row=2, 
    col=2
)

In [18]:
fig.update_layout(title_text="tlb access", 
                  bargroupgap=0.2,
                  height=800, width=1600,
                 )
fig.show()